In [1]:
# 安裝yahoo finance套件
!pip install yfinance

In [2]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間
import time  # 時間延遲套件，用於控制API呼叫頻率


# 爬取股票資訊的類別
class StockInfo():
    """
    股票基本資訊爬取類別

    功能：
    1. 從台灣證券交易所爬取所有上市公司的股號、股名、產業別
    2. 提供股號查詢股名的功能

    資料來源：台灣證券交易所 (TWSE) ISIN網站
    """

    # 取得全部股票的股號、股名
    def stock_name(self):
        """
        爬取所有台股的股號、股名、產業別

        Returns:
            DataFrame: 包含「股號」、「股名」、「產業別」三欄的資料表

        技術說明：
        - 使用requests取得網頁內容
        - 使用BeautifulSoup解析HTML
        - 過濾出4碼股號的上市公司（排除ETF等其他商品）
        """
        # 向台灣證券交易所發送請求
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        # 使用BeautifulSoup解析HTML內容
        url_data = BeautifulSoup(response.text, 'html.parser')
        # 找出所有的表格列（tr標籤）
        stock_company = url_data.find_all('tr')

        # 資料處理：提取股號、股名、產業別
        # \u3000 是全形空白字元，用來分隔股號和股名
        # 只保留4碼的股號（過濾掉ETF、特別股等）
        data = [
            (row.find_all('td')[0].text.split('\u3000')[0].strip(),
                row.find_all('td')[0].text.split('\u3000')[1],
                row.find_all('td')[4].text.strip())
            for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
        ]
        # 建立DataFrame並設定欄位名稱
        df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
        return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        """
        根據股號查詢股名

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            name_df (DataFrame): 包含股號和股名的資料表

        Returns:
            str: 股票名稱（例如：'台積電'）
        """
        return name_df.set_index('股號').loc[stock_id, '股名']

In [3]:
class StockAnalysis():
    """
    股票分析類別 - 整合股價資料與Gemini AI分析

    主要功能：
    1. 取得股票價格資料（技術面）
    2. 取得股票基本面資料（營收、EPS）
    3. 使用Gemini AI進行股票分析
    4. 多檔股票比較與推薦

    設計理念：
    - 整合多元數據源（價格、基本面）
    - 利用LLM進行智能分析
    """

    def __init__(self, gemini_api_key):
        """
        初始化股票分析系統

        Args:
            gemini_api_key (str): Google Gemini API金鑰

        初始化步驟：
        1. 設定Gemini API
        2. 建立生成模型實例
        """
        # 設定Gemini API金鑰
        genai.configure(api_key=gemini_api_key)
        """
        1. Gemini 2.5 Pro - 進階思考模型
        2. Gemini 2.5 Flash - 快速且智慧
        3. Gemini 2.5 Flash-Lite - 速度最快的 Flash 模型，可提高成本效益和輸送量。
        """
        # 使用Gemini 2.5 Pro模型（高階推理能力）
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        # 建立股票資訊物件
        self.stock_info = StockInfo()
        # 預先載入所有股票名稱（避免重複爬取）
        self.name_df = self.stock_info.stock_name()


    def stock_price(self, stock_id, days=15):
        """
        取得股票近期價格資料（技術面分析基礎）

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            days (int): 要取得的天數（預設15天）

        Returns:
            dict: 包含日期、收盤價、每日報酬率的字典

        技術說明：
        - 使用yfinance API取得Yahoo Finance資料
        - 計算每日報酬率 = (今日收盤-昨日收盤)/昨日收盤
        - 自動加上.TW後綴（台股代碼格式）
        """

        # 台股代碼需要加上.TW後綴
        stock_id += '.TW'
        # 計算日期區間
        end = dt.date.today() # 今天
        start = end - dt.timedelta(days=days) # 往前推N天

        # 禁用 FutureWarning
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)

        # 下載股價資料
        # auto_adjust=True 會自動調整除權息的價格
        df = yf.download(stock_id, start=start, auto_adjust=True)

        # 將欄位名稱改為中文（方便閱讀）
        df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
        # 建立回傳的資料結構
        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(), # 格式化日期
            '收盤價': df['收盤價'].tolist(), # 收盤價列表
            '每日報酬': df['收盤價'].pct_change().tolist() # 報酬率 = 價格變化百分比
        }
        return data


    def stock_fundamental(self, stock_id):
        """
        取得股票基本面資料（價值投資分析基礎）

        Args:
            stock_id (str): 股票代號

        Returns:
            dict: 包含季度日期、營收成長率、EPS、EPS季增率

        基本面指標說明：
        - 營收成長率：季度營收相比上一季的成長百分比
        - EPS (每股盈餘)：公司獲利能力的核心指標
        - EPS季增率：EPS相比上一季的成長百分比

        注意事項：
        - 某些股票可能無基本面資料（新上市或資料不全）
        - 使用try-except確保程式不會因單一股票錯誤而中斷
        """

        stock_id += '.TW'
        stock = yf.Ticker(stock_id)

        try:
            # 計算季度營收成長率
            # pct_change(-1)：與前一期（-1）比較的百分比變化
            # fill_method=None：不填補缺失值
            quarterly_revenue_growth = np.round(
                stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得季度EPS
            quarterly_eps = np.round(
                stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
                2
            )
            # 計算EPS季增率
            quarterly_eps_growth = np.round(
                stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得日期並格式化
            dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]
            # 組合資料（確保長度一致）
            data = {
                '季日期': dates[:len(quarterly_revenue_growth)],
                '營收成長率': quarterly_revenue_growth.tolist(),
                'EPS': quarterly_eps.tolist(),
                'EPS 季增率': quarterly_eps_growth.tolist()
            }
        except Exception as e:
            # 如果取得資料失敗，回傳空資料
            print(f"無法取得 {stock_id} 基本面資料: {e}")
            data = {'季日期': [], '營收成長率': [], 'EPS': [], 'EPS 季增率': []}

        return data

    def _get_reply(self, content_msg):
        """
        呼叫Gemini API並處理各種回應狀況（內部方法）

        Args:
            content_msg (str): 要傳送給Gemini的提示詞（prompt）

        Returns:
            str: Gemini的回應文字或錯誤訊息

        錯誤處理機制：
        1. finish_reason = 1 (STOP)：正常完成
        2. finish_reason = 2 (MAX_TOKENS)：超過長度限制
        3. finish_reason = 3 (SAFETY)：觸發安全政策
        4. finish_reason = 4 (RECITATION)：引用問題
        5. HTTP 429：API速率限制（免費版：每分鐘5次，每天25次）
        6. HTTP 503：服務暫時不可用
        """

        try:
            # 呼叫Gemini API生成內容
            response = self.model.generate_content(
                content_msg,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=10000,  # 最大輸出token數（避免過長）
                    temperature=1.0,  # 溫度參數（1.0 = 較有創意，0.0 = 較保守）
                )
            )

            # 檢查是否有回應候選
            if not response.candidates:
                return "模型未返回任何回應"

            candidate = response.candidates[0]
            finish_reason = candidate.finish_reason

            # finish_reason是整數值，代表不同的完成狀態：
            # 0 = UNSPECIFIED (未指定)
            # 1 = STOP (正常完成)
            # 2 = MAX_TOKENS (達到token限制)
            # 3 = SAFETY (安全過濾)
            # 4 = RECITATION (引用問題)
            # 5 = OTHER (其他原因)

            # 根據不同的完成原因處理
            if finish_reason == 1:  # STOP - 正常完成
                # 正常完成
                return candidate.content.parts[0].text

            elif finish_reason == 2:  # MAX_TOKENS - 達到長度限制
                try:
                    partial_text = candidate.content.parts[0].text if candidate.content.parts else ""
                    return f"{partial_text}\n\n [回應被截斷，已達到長度限制]"
                except:
                    return "回應超過長度限制。建議：1)增加 max_output_tokens 2)簡化 prompt"

            elif finish_reason == 3:  # SAFETY - 安全政策
                return "模型因安全政策無法提供回應"

            elif finish_reason == 4:  # RECITATION - 引用問題
                return "回應因引用問題被過濾"

            else:
                return f"未知完成原因: {finish_reason}"

        except Exception as e:
            # 處理API錯誤
            error_msg = str(e)
            # 速率限制錯誤（最常見）
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                return "達到 API 速率限制（免費層：5次/分鐘，25次/天）"
            # 服務暫時不可用
            elif "503" in error_msg:
                return "服務暫時不可用，請稍後重試"
            # 其他錯誤
            else:
                return f"API 錯誤: {error_msg}"


    def stock_gimini_analysis(self, stock_id):
        """
        使用Gemini AI分析單支股票（核心功能）

        Args:
            stock_id (str): 股票代號

        Returns:
            str: Gemini生成的股票分析報告

        分析流程：
        1. 取得股票名稱
        2. 收集技術面資料（價格、報酬率）
        3. 收集基本面資料（營收、EPS）
        4. 將資料摘要後傳送給Gemini
        5. 讓Gemini扮演金融分析師角色，產生專業分析

        Prompt設計要點：
        - 角色設定：明確告訴模型扮演「金融分析師」
        - 數據提供：精簡但完整的關鍵資訊
        - 輸出要求：技術面+基本面+風險評估
        - 語言指定：繁體中文
        """
        # 取得股票名稱
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 收集股價資料
        price_data = self.stock_price(stock_id)
        print('資料收集完畢')
        # 收集基本面資料
        stock_value_data = self.stock_fundamental(stock_id)

        # --- 資料摘要製作 ---
        # 目的：將大量資料濃縮成關鍵指標，避免超過token限制
        summary_parts = []

        # 股價摘要：最新價格 + 兩週漲跌幅
        if price_data and price_data['收盤價']:
            latest = price_data['收盤價'][-1] # 最新收盤價
            first = price_data['收盤價'][0] # 首日收盤價
            change = ((latest - first) / first) * 100 # 計算漲跌百分比
            summary_parts.append(f"股價：{latest:.2f} (兩週 {change:+.2f}%)")

        # 基本面摘要：最新一季的營收成長和EPS
        if stock_value_data and stock_value_data['季日期']:
            q = stock_value_data['季日期'][0] # 季度
            rev = stock_value_data['營收成長率'][0] # 營收成長率
            eps = stock_value_data['EPS'][0] # EPS
            summary_parts.append(f"基本面({q})：營收成長{rev*100:.1f}%, EPS {eps:.2f}")

        # 組合所有摘要
        combined = "；".join(summary_parts)

        # ============ 作業修改區 START ============
        # --- 建構Prompt ---
        # 這是Prompt Engineering的關鍵：如何引導LLM產生專業分析
        """
        'conservative': '''你是保守型財務顧問...
        1. 優先評估下檔風險
        2. 關注股息穩定性
        3. 避免高波動股票...''',

        'aggressive': '''你是成長型投資顧問...
        1. 關注營收成長動能
        2. 尋找突破訊號...''',

        'balanced': '''你是專業金融分析師...
        平衡評估機會與風險...'''
        """
        content_msg = f'''假設你是服務退休人士的財務顧問，請優先評估下檔風險和可能虧損，關注股息穩定性和財務健全度，避免推薦高波動或投機性股票，
用詞建議謹慎且需要注意風險，請根據以下股票數據生成投資建議{stock_name}。資料：{combined}
請提供詳細的技術面和基本面見解，並考慮風險因素。繁體中文回應。'''
        # ============ 作業修改區 END ============
        # 呼叫Gemini API
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_sort(self, message):
        """
        讓Gemini為多檔股票評分並排序

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini生成的評分排序結果

        評分標準：
        - 50分為基準
        - 正面因素加分（例如：營收成長、技術面轉強）
        - 負面因素扣分（例如：虧損、下跌趨勢）
        - 綜合技術面+基本面+市場環境

        應用場景：投資組合篩選、資金配置決策
        """

        content_msg = f'''你是專業股票分析師，根據趨勢報告評分(0-100)。
50分為基準，正面消息加分，負面消息扣分。
排序所有股票。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        """
        讓Gemini從多檔股票中選出最佳投資標的

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini選出的最佳股票及理由

        決策邏輯：
        - 即使所有選項都不理想，也要選出相對最佳的
        - 需要說明選擇理由（風險、報酬、時機等）
        - 模擬真實的投資決策情境

        教學價值：
        - 理解LLM如何進行多維度評估
        - 學習投資決策的思考框架
        """

        content_msg = f'''你是專業證券分析師，從趨勢報告中選出最適合投資的一檔股票。
即使都不理想也要選一檔，說明理由。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply

In [4]:
# 【重要】請修改為你自己的Gemini API Key
# ============ 作業修改區 START ============
gemini_api_key = 'APIKEY被警告需隱藏'
# ============ 作業修改區 END ============
# 建立股票分析物件
stock_analysis = StockAnalysis(gemini_api_key)

# 單支股價分析

In [5]:
# ============ 作業修改區 START ============
# 改成想分析的股票代號
reply = stock_analysis.stock_gimini_analysis('0050')
# ============ 作業修改區 END ============
print(reply)

[*********************100%***********************]  1 of 1 completed


資料收集完畢
無法取得 0050.TW 基本面資料: 'Total Revenue'
好的，王先生/王太太，您好。

很高興能為您分析元大台灣50（0050）這檔ETF。考量到您已屆退休，我們的首要目標是**保護本金、追求穩健的現金流，並嚴格控制下檔風險**。我會以最審慎的角度，為您剖析這項投資標的。

請您理解，以下的分析是基於公開資訊和我個人的專業判斷，任何投資都存在風險，最終決策仍需考量您自身的財務狀況與風險承受能力。

---

### **投資建議總覽**

**總體評估：**
元大台灣50（0050）是一檔追蹤台灣市值前50大公司的指數股票型基金（ETF）。對退休人士而言，它具備**分散風險、成分股體質穩健、管理費用低廉**的特性，可視為**退休資產配置中「股票部位」的核心選項之一**。然而，它本質上仍是100%的股票投資，無法完全規避市場的系統性風險，因此**不建議將所有資金單獨投入**。

**核心觀點：**
*   **優點：** 省去研究個股的麻煩，一次性投資台灣最具競爭力的龍頭企業，享有其長期成長與配息的果實。
*   **缺點：** 股價會隨台股大盤波動，短期內可能出現虧損。其產業集中於電子業（特別是半導體），使表現與單一產業景氣高度相關。

---

### **一、 基本面分析 (為何值得考慮？)**

基本面是我們評估一項資產是否值得長期持有的基石，特別對於退休規劃而言，這比短期的股價波動更為重要。

1.  **成分股的財務健全度 (Financial Soundness):**
    *   0050的成分股是台灣市值最大、流動性最佳的50家公司，例如台積電、聯發科、鴻海、富邦金等。
    *   這些公司通常是產業龍頭，擁有強大的護城河、穩定的營收和獲利能力。相較於中小型公司，它們在經濟逆風時的抵抗能力更強，倒閉風險極低。這點完全符合我們對**財務健全度**的要求。

2.  **股息穩定性 (Dividend Stability):**
    *   0050每年會進行兩次除息（通常在1月和7月）。其股息來自於這50家公司發放的現金股利。
    *   歷史數據顯示，除了在重大金融危機（如2008年）的次一年度股息可能下降外，0050的配息紀錄相當穩定。因為即使部分公司當年營運不佳，其他公司穩健的獲利仍能支撐整體的股息發

# 收集多檔股票的趨勢報告

In [6]:
# 建立股票清單
# ============ 作業修改區 START ============
# 請修改為你想分析的股票清單（建議3-5檔，避免超過API限制）
# 台股股票參考：
# 2330 台積電, 2317 鴻海, 2454 聯發科, 2412 中華電, 2881 富邦金
# 2382 廣達, 2303 聯電, 3711 日月光投控, 2891 中信金, 2886 兆豐金
# 2884 玉山金，2890 永豐金，2891，中信金
stock_list = ['2884', '2890', '2891']
# ============ 作業修改區 END ============

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

print(f"\n{'='*70}")
print(f"開始分析 {len(stock_list)} 檔股票")
print(f"{'='*70}\n")

# 批次處理每一檔股票
for i, stock in enumerate(stock_list, 1):
    print(f"[{i}/{len(stock_list)}] 處理股票: {stock}")

    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        try:
            # 分析股票
            print(f"   正在分析...")
            analysis = stock_analysis.stock_gimini_analysis(stock_id=stock)

            # 儲存結果
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(analysis)

            content[stock] = analysis
            print(f"   分析完成並已儲存\n")

            # 速率限制控制
            if i < len(stock_list):
                wait_time = 12
                print(f"   等待 {wait_time} 秒以符合 API 速率限制...\n")
                time.sleep(wait_time)

        except Exception as e:
            error_msg = f"錯誤: {str(e)}"
            print(f"   {error_msg}\n")
            content[stock] = error_msg

    else:
        # 讀取已存在的分析
        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()
        print(f"   從快取讀取\n")

print(f"{'='*70}")
print("所有股票分析完成！")
print(f"{'='*70}\n")

print("\n" + "="*70)
print("分析結果")
print("="*70 + "\n")

for stock_id, analysis in content.items():
    stock_name = stock_analysis.stock_info.get_stock_name(stock_id, stock_analysis.name_df)

    print("="*70)
    print(f"【{stock_id} - {stock_name}】")
    print("="*70)
    print(analysis)
    print("\n")

[*********************100%***********************]  1 of 1 completed


開始分析 3 檔股票

[1/3] 處理股票: 2884
   正在分析...
資料收集完畢


   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...



[*********************100%***********************]  1 of 1 completed

[2/3] 處理股票: 2890
   正在分析...
資料收集完畢


   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...



[*********************100%***********************]  1 of 1 completed

[3/3] 處理股票: 2891
   正在分析...
資料收集完畢


   分析完成並已儲存

所有股票分析完成！


分析結果

【2884 - 玉山金】
好的，收到您的指示。我將以一位服務退休人士的財務顧問身份，為您提供一份關於玉山金(2884)的審慎投資評估。

在我們的分析中，**首要考量是資本的保全與可預測的現金流**，而非追求短期的高報酬。因此，我會將重點放在下檔風險、股息的穩定性及公司的長期營運體質上。

---

### **玉山金融控股 (2884) 投資評估報告**

**報告日期：** 2024年5月23日
**目標對象：** 尋求穩健收益與資本保值的退休人士

#### **總結建議**

整體而言，玉山金是一家體質健全、經營穩健的優質金融機構，其在數位金融與財富管理領域的領先地位，使其具備長期增長潛力。對退休人士而言，它可被視為**長期核心資產配置的一部分**。

然而，基於目前的股價位階與過去一兩年股息政策的變動，我對此標的的建議是「**審慎樂觀，建議採取分批佈局策略，不宜一次性重壓**」。這意味著，它值得您考慮納入投資組合，但應謹慎控制投入的資金比例與進場時機。

---

#### **一、 基本面見解 (著重於穩定性與風險)**

1.  **財務健全度與經營穩健性 (正面)**
    *   玉山金長期以來是金融業的模範生，資本適足率、逾期放款率等關鍵指標均維持在優異水準，顯示其資產品質良好，風險控管嚴謹。這對於厭惡風險的退休投資人而言，是最重要的基石。
    *   公司在信用卡、財富管理及中小企業放款等業務上具有市場領先地位，營收來源相對多元，有助於抵禦單一業務的景氣波動。

2.  **股息穩定性與未來展望 (中性偏正面，但需留意)**
    *   **過去的挑戰：** 2023年，受到美國急速升息導致債券投資部位出現評價損失的影響，玉山金的股利政策改為以股票股利為主、現金股利為輔。這對於**高度依賴現金股息作為退休生活費來源的投資人**，構成了一定的不確定性。這是您必須正視的**首要風險**。
    *   **未來的展望：** 隨著利率環境趨於穩定，以及公司獲利能力回升，市場普遍預期其現金股利發放能力將在未來一至兩年內逐步恢復正常水準。您提供的預估數據「營收成長14.0%, EPS 0.58」雖未註明是單季或全年預測，但**營收成長14.0%**這個數字本身傳達了正面的營運訊號，

# 推薦一檔股票

In [7]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

好的，身為專業證券分析師，我將根據您提供的三份趨勢報告，進行綜合評估與比較，並選出在當前資訊基礎下，相對最適合投資的一檔股票。

### **最終選擇**

經過綜合評估三份趨勢報告，我選擇 **2884 玉山金** 作為最適合投資的標的。

### **核心決策理由**

我的決策核心基於**「規避明確的未來衰退風險」**。在投資決策中，預期性的利空消息，其殺傷力遠大於已知的、暫時性的利空。

1.  **玉山金 (2884) - 唯一具備正面成長預期：**
    在三份報告中，玉山金是**唯一**提供正面未來基本面預測（**營收成長14.0%**）的公司。儘管報告中提及了評價偏高、以及過去一年股利政策不穩定的問題，但「營收成長」是企業獲利的火車頭，代表其本業營運依然處於擴張軌道。股利政策的問題主要是由去年全球利率急升的特殊環境造成，隨著獲利回升，市場普遍預期其現金股利發放能力將逐步恢復。因此，玉山金面臨的是「評價修正」與「股利恢復不如預期」的風險，但其營運前景本身是正向的。

2.  **永豐金 (2890) & 中信金 (2891) - 面臨明確的衰退預警：**
    *   **永豐金 (2890)** 的報告明確指出，2025年預估**營收將衰退9.0%**，EPS也將顯著下滑。
    *   **中信金 (2891)** 的預測更為嚴峻，2025年預估**營收將衰退45.0%**。

    對於任何分析師而言，一個明確的「營收衰退」預測都是最嚴重的警訊。這直接威脅到未來的EPS、股利發放能力，以及市場對其的評價。更何況，這兩檔股票的股價目前都處於歷史相對高檔區。**在高價位買入一檔未來營運可能衰退的股票，是投資中的大忌**，這意味著投資人可能面臨「基本面下修」與「股價回檔」的雙殺風險。

**簡單來說，我的選擇邏輯如下：**

| 股票代號 | 未來基本面預測 | 主要風險 | 決策 |
| :--- | :--- | :--- | :--- |
| **2884 玉山金** | **營收成長 +14.0%** | 評價偏高、股利政策不確定性 | **選擇** (前景正向，風險可控) |
| **2890 永豐金** | 營收衰退 -9.0% | 基本面衰退、股價高檔 | **排除** (風險/報酬比不佳) |
| **2891 中信金** | 

# 推薦股票的評分排序

In [8]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

好的，我將以專業股票分析師的角色，根據您提供的趨勢報告內容，對每支股票進行評分與排序。

評分基準為50分，代表中性觀察。正面消息將拉高分數，負面消息或風險提示則會拉低分數。

---

### **股票趨勢報告評分**

#### **1. 玉山金 (2884)**

*   **評分：75 / 100**

*   **核心摘要：**
    *   **加分項 (+)**：報告明確指出玉山金為「體質健全、經營穩健的優質金融機構」，財務健全度與風險控管被評為「正面」。最關鍵的加分來自於「營收成長14.0%」的正面營運訊號，顯示本業動能強勁，為未來股息發放提供了堅實基礎。長期趨勢被視為「緩步向上」，適合長期持有。
    *   **扣分項 (-)**：主要風險在於「估值略有偏高」，導致安全邊際較低。此外，2023年股息政策的變動對依賴現金流的投資人構成「不確定性」，被列為首要風險，這兩點是主要的扣分來源。

*   **綜合評語：** 報告整體基調為「審慎樂觀」。儘管存在估值偏高與股息政策的不確定性，但強勁的營收成長預期與優良的公司體質，使其在本次評估中獲得最高分。

---

#### **2. 永豐金 (2890)**

*   **評分：25 / 100**

*   **核心摘要：**
    *   **加分項 (+)**：報告肯定其過往的「股息穩定性」與「財務健全度」，業務多元化也提供了風險分散的優點。這些是金融存股的基本盤，給予基礎分數。
    *   **扣分項 (-)**：報告中最關鍵的負面因素是「**未來營收預測衰退 (-9.0%)**」，這被標示為「**最關鍵的警訊**」，直接衝擊未來的獲利與股利發放能力，導致評分被大幅拉低。此外，股價處於「歷史相對高的價位區間」，技術面「缺乏上攻動能」，追高風險與評價修正風險顯著，進一步侵蝕了分數。

*   **綜合評語：** 報告基調為「中性偏向審慎」，並強烈建議「暫不追高」。未來的營收衰退預期是致命傷，完全蓋過了其過往穩健的表現。在風險遠大於潛在回報的情況下，得分偏低。

---

#### **3. 中信金 (2891)**

*   **評分：15 / 100**

*   **核心摘要：**
    *   **加分項 (+)**：報告承認其本質上是「體質健全、營運多元、長期配息」的好公司，